<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [5]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)
np.unique(y_train_smt, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_smt, y_train_smt),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
! pip install dagshub

In [9]:
import dagshub
dagshub.init(repo_owner='bhavanareddy19', repo_name='mlflow_dagshub_demo', mlflow=True)



Accessing as bhavanareddy19

Initialized MLflow to track repo "bhavanareddy19/mlflow_dagshub_demo"

Repository bhavanareddy19/mlflow_dagshub_demo initialized!

In [10]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [11]:
# Initialize MLflow
mlflow.set_experiment("Anomalys")
mlflow.set_tracking_uri("https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/12/18 23:23:33 INFO mlflow.tracking.fluent: Experiment with name 'Anomalys' does not exist. Creating a new experiment.
2025/12/18 23:23:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/e1702a86e12947f48cda2ebd12ed0dbb
🧪 View experiment at: https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow/#/experiments/0


2025/12/18 23:23:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/442f5ef5762d40929ae29cfeb3ccdfab
🧪 View experiment at: https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow/#/experiments/0


2025/12/18 23:24:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/59278795221e41b09b97085e649f2821
🧪 View experiment at: https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow/#/experiments/0


2025/12/18 23:24:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/f3d78551bfe24d6e93db465d4a5e6345
🧪 View experiment at: https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow/#/experiments/0


### Register the Model

In [12]:
import mlflow
print(mlflow.get_tracking_uri())

https://dagshub.com/bhavanareddy19/mlflow_dagshub_demo.mlflow


In [13]:
run_id = "e4450974dc9346b2a57f6bcebc4e1a4f"
model_uri = f'runs:/{run_id}/model'  # ✅ Changed from "model_name" to "model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name="XGBClassifier_SMOTE")

RestException: RESOURCE_DOES_NOT_EXIST: Run not found

### Load the Model

In [ ]:
model_name = "XGBClassifier_SMOTE"  # Underscores, not spaces
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [ ]:
model_name = "XGBClassifier_SMOTE"  # Correct name with underscores
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGBClassifier_SMOTE' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1766123149717, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1766123149717, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='e4450974dc9346b2a57f6bcebc4e1a4f', run_link='', source='models:/XGBClassifier_SMOTE/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [ ]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 